In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from rff.layers import GaussianEncoding #pip install random-fourier-features-pytorch
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
import os
import optuna
from optuna.trial import TrialState
from tab_transformer_pytorch import TabTransformer
import time

c:\Users\prime\anaconda3\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Run regardless if you do or do not have GPU so all tensors are moved to right location later on
if torch.cuda.is_available():
    device_in_use = torch.device("cuda")
    print("GPU is available and being used")
else:
    device_in_use = torch.device("cpu")
    print("GPU is not available, using CPU instead")

GPU is not available, using CPU instead


# LOAD AND PROCESS DATA
**EXAMPLE WITH Income**

In [3]:
df_train = pd.read_csv('../../datasets/higgs/train.csv')
df_test = pd.read_csv('../../datasets/higgs/test.csv')
df_val = pd.read_csv('../../datasets/higgs/validation.csv') #READ FROM RIGHT SPOT

df_train.columns

Index(['class', 'lepton_pT', 'lepton_eta', 'lepton_phi',
       'missing_energy_magnitude', 'missing_energy_phi', 'jet1pt', 'jet1eta',
       'jet1phi', 'jet1b-tag', 'jet2pt', 'jet2eta', 'jet2phi', 'jet2b-tag',
       'jet3pt', 'jet3eta', 'jet3phi', 'jet3b-tag', 'jet4pt', 'jet4eta',
       'jet4phi', 'jet4b-tag', 'm_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb',
       'm_wbb', 'm_wwbb'],
      dtype='object')

In [4]:
def categorize_columns(dataframe, target):
    categorical_columns = []
    continuous_columns = []
    unique_classes_per_column = []  # To hold the number of unique classes for each categorical column

    for column in dataframe.columns:
        if dataframe[column].dtype == 'object' or len(dataframe[column].unique()) <= 10:
            # If the column's data type is 'object' or it has 10 or fewer unique values, consider it categorical.
            if column != target:
                categorical_columns.append(column)
                unique_classes_per_column.append(dataframe[column].nunique())  # Store the number of unique classes
        else:
            # Otherwise, consider it continuous.
            continuous_columns.append(column)

    return categorical_columns, continuous_columns, unique_classes_per_column

In [5]:
target = 'class'
cat_columns, cont_columns, unique_classes_per_column = categorize_columns(df_train, target)

if target in cont_columns:
    cont_columns.remove(target)
elif target in cat_columns:
    print("in here")
    cat_columns.remove(target)

print("Target:", target)
print("Categorical: ", cat_columns)
print("Continous: ", cont_columns)
print("Unique Classes per Column: ", unique_classes_per_column)

#CHECKING TO MAKE SURE YOUR LIST IS CORRECT (NO NEED TO TOUCH)
yourlist = cat_columns + cont_columns + [target]
yourlist.sort()
oglist = list(df_train.columns)
oglist.sort()

assert(yourlist == oglist), "You may of spelled feature name wrong or you forgot to put on of them in the list"

target_classes = [max(len(df_train[target].value_counts()), len(df_val[target].value_counts()),len(df_test[target].value_counts()))]
print(target_classes)
# Create a StandardScaler and fit it to the cont features
scaler = StandardScaler()
scaler.fit(df_train[cont_columns])

# Transform the training, test, and validation datasets
df_train[cont_columns] = scaler.transform(df_train[cont_columns])
df_test[cont_columns] = scaler.transform(df_test[cont_columns])
df_val[cont_columns] = scaler.transform(df_val[cont_columns])

# label encode target...
le = LabelEncoder()
df_train[target] = le.fit_transform(df_train[target])
df_test[target] = le.fit_transform(df_test[target])
df_val[target] = le.fit_transform(df_val[target])

# ...and categorical features
for feature in cat_columns:
    print(feature)
    df_train[feature] = le.fit_transform(df_train[feature])
    df_test[feature] = le.fit_transform(df_test[feature])
    df_val[feature] = le.fit_transform(df_val[feature])

print(df_train['jet1b-tag'].value_counts())

class SingleTaskDataset(Dataset):
    def __init__(self, df : pd.DataFrame, cat_columns, num_columns,task1_column):
        self.n = df.shape[0]
        
        self.task1_labels = df[task1_column].astype(np.float32).values

        self.cate = df[cat_columns].astype(np.int64).values
        self.num = df[num_columns].astype(np.float32).values


    def __len__(self):
        return self.n
    
    def __getitem__(self, idx):
        # Retrieve features and labels from the dataframe using column names
        cat_features = self.cate[idx]
        num_features = self.num[idx]
        labels_task1 = self.task1_labels[idx]

        return cat_features, num_features, labels_task1

#Wrapping in Dataset
train_dataset = SingleTaskDataset(df_train, cat_columns, cont_columns, 'class')
val_dataset = SingleTaskDataset(df_val, cat_columns, cont_columns, 'class')
test_dataset = SingleTaskDataset(df_test, cat_columns, cont_columns, 'class')

#This is a hyperparameter that is not tuned. Maybe mess with what makes sense here
#Also try looking to see what other papers have done
batch_size = 256

# Wrapping with DataLoader for easy batch extraction
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

Target: class
Categorical:  ['jet1b-tag', 'jet2b-tag', 'jet3b-tag', 'jet4b-tag']
Continous:  ['lepton_pT', 'lepton_eta', 'lepton_phi', 'missing_energy_magnitude', 'missing_energy_phi', 'jet1pt', 'jet1eta', 'jet1phi', 'jet2pt', 'jet2eta', 'jet2phi', 'jet3pt', 'jet3eta', 'jet3phi', 'jet4pt', 'jet4eta', 'jet4phi', 'm_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb', 'm_wbb', 'm_wwbb']
Unique Classes per Column:  [3, 3, 3, 3]
[2]
jet1b-tag
jet2b-tag
jet3b-tag
jet4b-tag
jet1b-tag
0    33704
2    28139
1     6791
Name: count, dtype: int64


In [6]:
# each task loss is scaled by its own learnable parameter, then regularization is applied 
class LossFunctions(nn.Module):
    def __init__(self, num_tasks):
        super(LossFunctions, self).__init__()
        self.num_tasks = num_tasks

        self.loss_fns = [nn.CrossEntropyLoss() for x in range(num_tasks)] 

    def forward(self, predictions, labels_task1):

        #task 1
        target = labels_task1.long()
        prediction = predictions
        loss_fn = self.loss_fns[0]
        task_loss = loss_fn(prediction, target)
        
        return task_loss
    
# Training and Testing Loops
def train(dataloader, model, loss_function, optimizer, device_in_use):
    model.train()

    total_loss = 0
    total_correct_1 = 0
    total_samples_1 = 0
    all_targets_1 = []
    all_predictions_1 = []

    for (categorical, numerical, labels_task1) in dataloader:
        categorical, numerical, labels_task1 = categorical.to(device_in_use), numerical.to(device_in_use), labels_task1.to(device_in_use)
        
        #running them through model and modifying the shape slightly for the loss function
        task_predictions = model(categorical, numerical)
        task_predictions = task_predictions.squeeze(1)
        
        loss = loss_function(task_predictions, labels_task1)
        total_loss += loss.item()
        
        #computing accuracy for first target
        y_pred_softmax_1 = torch.softmax(task_predictions, dim=1)
        _, y_pred_labels_1 = torch.max(y_pred_softmax_1, dim=1)
        total_correct_1 += (y_pred_labels_1 == labels_task1).sum().item()
        total_samples_1 += labels_task1.size(0)
        all_targets_1.extend(labels_task1.cpu().numpy())
        all_predictions_1.extend(y_pred_labels_1.cpu().numpy())


        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = total_loss/len(dataloader)
    accuracy_1 = total_correct_1 / total_samples_1

    return avg_loss, accuracy_1

def test(dataloader, model, loss_function, device_in_use):
    model.eval()
    total_loss = 0
    
    total_loss = 0
    total_correct_1 = 0
    total_samples_1 = 0
    all_targets_1 = []
    all_predictions_1 = []

    with torch.no_grad():
        for (categorical, numerical, labels_task1) in dataloader:
            categorical, numerical, labels_task1 = categorical.to(device_in_use), numerical.to(device_in_use), labels_task1.to(device_in_use)
            
            #running them through model and modifying the shape slightly for the loss function
            task_predictions = model(categorical, numerical)
            task_predictions = task_predictions.squeeze(1)
            loss = loss_function(task_predictions, labels_task1)
            total_loss += loss.item()
            
            #computing accuracy for first target
            y_pred_softmax_1 = torch.softmax(task_predictions, dim=1)
            _, y_pred_labels_1 = torch.max(y_pred_softmax_1, dim=1)
            total_correct_1 += (y_pred_labels_1 == labels_task1).sum().item()
            total_samples_1 += labels_task1.size(0)
            all_targets_1.extend(labels_task1.cpu().numpy())
            all_predictions_1.extend(y_pred_labels_1.cpu().numpy())

    avg = total_loss/len(dataloader)
    accuracy_1 = total_correct_1 / total_samples_1
    f1_1 = f1_score(all_targets_1, all_predictions_1, average='weighted')

    return avg, accuracy_1, all_predictions_1, all_targets_1, f1_1

def format_metric(value): # Used to format the metrics output
    return f"{value:.4f}"


In [7]:
#Testing against the test dataset

# hidden (embedding)
#dimension, the number of layers and the number of 
#attention heads are fixed to 32, 6, and 8 respectively. The MLP
#layer sizes are set to {4 × l, 2 × l}, where l is the size of its input.
model = TabTransformer(categories=tuple(unique_classes_per_column),
                       num_continuous=len(cont_columns),
                       dim = 32,                           # dimension, paper set at 32
                        dim_out = target_classes[0],                        # binary prediction, but could be anything
                        depth = 6,                          # depth, paper recommended 6
                        heads = 8,                          # heads, paper recommends 8
                        attn_dropout = 0.1,                 # post-attention dropout
                        ff_dropout = 0.1,                   # feed forward dropout
                        mlp_hidden_mults = (4, 2),          # relative multiples of each hidden dimension of the last mlp to logits
                        mlp_act = nn.ReLU(),    
                       ).to(device_in_use)
loss_functions = LossFunctions(1)
optimizer = torch.optim.AdamW(params=model.parameters(), lr = 0.0001, weight_decay=0.000001)
epochs = 100

train_losses = []
train_accuracies_1 = [] 

test_losses = []
test_accuracies_1 = []
  
test_f1_scores = [] 
all_attention_scores = []

#Time will be recorded for all 100 epochs - This means the results will not be comparable to Xgboost but that is ok, we will only compare between transformer models who will also train for 100 epochs
start_time = time.process_time()

for t in range(epochs):
  train_loss, train_accuracy_1 = train(train_dataloader, model, loss_functions, optimizer, device_in_use=device_in_use)
  test_loss, test_accuracy_1, all_predictions_1, all_targets_1, f1_1 = test(test_dataloader, model, loss_functions, device_in_use=device_in_use)
  train_losses.append(train_loss)
  train_accuracies_1.append(train_accuracy_1)
  # train_accuracies_2.append(train_accuracy_2)
  # train_recalls.append(train_recall) 
  # train_f1_scores.append(train_f1)
  test_losses.append(test_loss)
  test_accuracies_1.append(test_accuracy_1)
  # test_accuracies_2.append(test_accuracy_2)
  # test_recalls.append(test_recall)
  test_f1_scores.append(f1_1)
  # Formatting for easier reading
  epoch_str = f"Epoch [{t+1:2}/{epochs}]"
  train_metrics = f"Train: Loss {format_metric(train_loss)}, Accuracy {format_metric(train_accuracy_1)}"
  test_metrics = f"Test: Loss {format_metric(test_loss)}, Accuracy {format_metric(test_accuracy_1)}, F1 {format_metric(f1_1)}"
  print(f"{epoch_str:20} | {train_metrics:65} | {test_metrics}")

total_time = time.process_time() - start_time

# Plotting the loss curves
plt.figure(figsize=(15, 5))
plt.subplot(1, 1, 1)
plt.plot(range(1, epochs+1), train_losses, label='Train Loss')
plt.plot(range(1, epochs+1), [l for l in test_losses], label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss Curve')
plt.legend()

# Display confusion matrix for the first task (Traffic Type) on test data
conf_matrix_1 = confusion_matrix(all_targets_1, all_predictions_1)
print("Confusion Matrix")
print(conf_matrix_1)

best_index = test_accuracies_1.index(max(test_accuracies_1))
print(f"Best accuracy {test_accuracies_1[best_index]}")
print(f"Best F1 {test_f1_scores[best_index]}")
print(f"100 epochs of training and evaluation took, {total_time}")

Epoch [ 1/100]       | Train: Loss 0.6786, Accuracy 0.5682                               | Test: Loss 0.6716, Accuracy 0.5897, F1 0.5890
Epoch [ 2/100]       | Train: Loss 0.6581, Accuracy 0.6120                               | Test: Loss 0.6577, Accuracy 0.6061, F1 0.5797
Epoch [ 3/100]       | Train: Loss 0.6434, Accuracy 0.6291                               | Test: Loss 0.6445, Accuracy 0.6269, F1 0.6143
Epoch [ 4/100]       | Train: Loss 0.6363, Accuracy 0.6367                               | Test: Loss 0.6380, Accuracy 0.6350, F1 0.6348
Epoch [ 5/100]       | Train: Loss 0.6314, Accuracy 0.6432                               | Test: Loss 0.6327, Accuracy 0.6414, F1 0.6365
Epoch [ 6/100]       | Train: Loss 0.6286, Accuracy 0.6476                               | Test: Loss 0.6292, Accuracy 0.6447, F1 0.6433
Epoch [ 7/100]       | Train: Loss 0.6243, Accuracy 0.6524                               | Test: Loss 0.6269, Accuracy 0.6481, F1 0.6408


KeyboardInterrupt: 